<a href="https://colab.research.google.com/github/halona719halona/datasets/blob/main/image2recipe_data_load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!pip install bs4
!pip install lxml


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# New Section

In [2]:
!pwd

/content


In [60]:
import datetime as dt
import csv
from bs4 import BeautifulSoup
from lxml import html
from datetime import datetime
from multiprocessing import Pool
from random import uniform, choice
from time import sleep, time
import requests
import os.path
import re
import os
import pandas as pd
import urllib.request

In [9]:
csv_file = "/content/sample_data/chefkoch_rezepte_27-12-2017.csv"

In [36]:
df = pd.read_csv(csv_file)

In [37]:
df.head(5)

,idx,recipe_id,recipe_name,average_rating,stars_shown,votes,difficulty,preparation_time,date,link,has_picture
0,0,recipe-3449791514066439,Bratapfel-Rotkohl,0 Bewertungen - Ø 2.5,star-unb,0.0,normal,45 min.,24.12.2017,https://www.chefkoch.de/rezepte/34497915140664...,no
1,1,recipe-3449721514056932,„Frühstückswolke 7“ aus Fiefhusen,0 Bewertungen - Ø 2.5,star-unb,0.0,pfiffig,15 min.,24.12.2017,https://www.chefkoch.de/rezepte/34497215140569...,no
2,2,recipe-3449701514054063,Deichgrafensalat von der Westküste,0 Bewertungen - Ø 2.5,star-unb,0.0,normal,30 min.,24.12.2017,https://www.chefkoch.de/rezepte/34497015140540...,no
3,3,recipe-3449651514047893,Geschichteter Kohl,0 Bewertungen - Ø 2.5,star-unb,0.0,normal,40 min.,24.12.2017,https://www.chefkoch.de/rezepte/34496515140478...,no
4,4,recipe-3449681514052511,Rinderlendenragout,0 Bewertungen - Ø 2.5,star-unb,0.0,normal,15 min.,24.12.2017,https://www.chefkoch.de/rezepte/34496815140525...,no


In [38]:
failed_urls = []

In [39]:
desktop_agents = ['Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/602.2.14 (KHTML, like Gecko) Version/10.0.1 Safari/602.2.14',
                 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36',
                 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.98 Safari/537.36',
                 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.98 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0']

def random_headers():
    return {'User-Agent': choice(desktop_agents),'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'}

In [57]:
def get_html(url):
    i = 5
    while i > 0:
        try:
            page = requests.get(url, headers=random_headers())
            if page.status_code != requests.codes.ok:
                page.raise_for_status()
            else:
#                 print(url)
                return page.text        
        except requests.exceptions.RequestException as e:
            print("Could not fetch " + url)
            print(e)
            # sichere url zu einer Liste von fehlgeschlagenen Links
            failed_urls.append(url)
            sleep(5)
            i = i - 1
            continue

    print(url)
    return None
def get_ingredients(soup):
    # liste von zubereitungen
    ingredient_list = []
    amounts_ingredients = soup.find('table', class_="ingredients table-header").find_all('tr')

    for tr in amounts_ingredients:
        am = tr.find_all('td')[0].text.strip().replace(" ","")  # amount
        td = tr.find_all('td')[1].text.strip().split(',')[0] # ingredient
        td = re.sub('\(.*?\)','', td)
        ingr = am + '@' + td
        ingredient_list.append(ingr)
        
    return(str.join(',', ingredient_list))

def get_tags(soup):
    tags = []
    tag_cloud = soup.find('ul', class_ = "r-nav-box r-std").find_all('li')
    for li in tag_cloud:
        tags.append(li.find('a').text.strip())
        
    return(str.join(',', tags))
def get_recipe_info(url):
    sleep_time = uniform(2, 4)
    sleep(sleep_time)
    
    html_text = get_html(url)
    try:
        soup = BeautifulSoup(html_text, 'lxml')

        # tags ingredients preparation
        ingredient_list = get_ingredients(soup)
        tags = get_tags(soup)
        # prep = get_zubereitung(soup)
        
        # pics
        list_pics = get_n_pictures(html_text)
        # n_pics = len(list_pics)
        
        # write dictionary
        data = {'link' : url,
                'ingredients' : ingredient_list,
                # 'zubereitung' : prep,
                'tags': tags, 
                'image_link': list_pics}
        
    except:
        data = {}
        print('error')
    return data

In [65]:

def get_n_pictures(html_text):
  tree = html.fromstring(html_text)
  # alle bilder links
  links_pics = tree.xpath(
      '//div[contains(@class, "ds-mb-left recipe-image")] \
      //img[@class="i-amphtml-fill-content i-amphtml-replaced-content"]/@src')
  return links_pics[0]

In [67]:
dd = df[:100000]

In [68]:
dd['image_link'] = ''
dd['ingredients'] = ''
dd['tags'] = ''

<ipython-input-68-b9e0847e784d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dd['image_link'] = ''
<ipython-input-68-b9e0847e784d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dd['ingredients'] = ''
<ipython-input-68-b9e0847e784d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

In [52]:
dd.head()

,idx,recipe_id,recipe_name,average_rating,stars_shown,votes,difficulty,preparation_time,date,link,has_picture,image_link,ingredients,tags
0,0,recipe-3449791514066439,Bratapfel-Rotkohl,0 Bewertungen - Ø 2.5,star-unb,0.0,normal,45 min.,24.12.2017,https://www.chefkoch.de/rezepte/34497915140664...,no,,,
1,1,recipe-3449721514056932,„Frühstückswolke 7“ aus Fiefhusen,0 Bewertungen - Ø 2.5,star-unb,0.0,pfiffig,15 min.,24.12.2017,https://www.chefkoch.de/rezepte/34497215140569...,no,,,
2,2,recipe-3449701514054063,Deichgrafensalat von der Westküste,0 Bewertungen - Ø 2.5,star-unb,0.0,normal,30 min.,24.12.2017,https://www.chefkoch.de/rezepte/34497015140540...,no,,,
3,3,recipe-3449651514047893,Geschichteter Kohl,0 Bewertungen - Ø 2.5,star-unb,0.0,normal,40 min.,24.12.2017,https://www.chefkoch.de/rezepte/34496515140478...,no,,,
4,4,recipe-3449681514052511,Rinderlendenragout,0 Bewertungen - Ø 2.5,star-unb,0.0,normal,15 min.,24.12.2017,https://www.chefkoch.de/rezepte/34496815140525...,no,,,


In [62]:
save_path = "/content/images"
os.makedirs(save_path, exist_ok=True)

In [ ]:
for i, row in dd.iterrows():
  url = row["link"]
  data = get_recipe_info(url)
  print(data)
  if len(data) > 0:
    dd.loc[i, 'image_link'] = data['image_link']
    dd.loc[i, 'ingredients'] = data['ingredients']
    dd.loc[i, 'tags'] = data['tags']
    img_name = os.path.join(save_path, row['recipe_id']+'.jpg')
    urllib.request.urlretrieve(str(data['image_link']), str(img_name))
dd.to_csv("/content/recipe_data.csv", index=False)
  


{'link': 'https://www.chefkoch.de/rezepte/3449791514066439/Bratapfel-Rotkohl.html', 'ingredients': '2½kg@Rotkohl ,750g@Äpfel ,500g@Zwiebel,1¼Liter@Apfelsaft,2m.-große@Zitrone,1EL,gestr.@Paprikapulver,1EL,gestr.@Paprikapulver,3EL,gestr.@Honig,3EL,gestr.@Salz,1EL@Pfefferkörner,1TL@Pimentkörner,1TL@Wacholderbeere,2@Lorbeerblätter,evtl.@Salz und Pfeffer,evtl.@Zucker,evtl.@Essigessenz', 'tags': 'Rezepte finden\ue916,Kategorien,Rezepte eingeben', 'image_link': 'https://img.chefkoch-cdn.de/rezepte/3449791514066439/bilder/1314914/crop-360x240/bratapfel-rotkohl.jpg'}
{'link': 'https://www.chefkoch.de/rezepte/3449721514056932/Fruehstueckswolke-7-aus-Fiefhusen.html', 'ingredients': '2@Ei,1EL@Parmesan,2Scheibe/n@Frühstücksspeck,etwas@Rapsöl,1EL,gehäuft@Lauchzwiebel,@Meersalz und Pfeffer', 'tags': 'Rezepte finden\ue916,Kategorien,Rezepte eingeben', 'image_link': 'https://img.chefkoch-cdn.de/rezepte/3449721514056932/bilder/1079238/crop-360x240/fruehstueckswolke-7-aus-fiefhusen.jpg'}
{'link': 'https: